# Imports

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F


# Data Loading

In [ ]:
# from torch.utils.data import Dataset

# class imageDataset(Dataset):
#     #Subclass datasets in order to create a custom one that i dont have to load everything in till im ready
    
#     #All neeccesary methods to implement: Inits it with data
#     def __init__(self, folder_path):
#         self.data = #Stuff from folder path

#     def __len__(self):
#         return len(self.data)
    
#     def __getitem__(self, idx):
#         #Gets the item at idx in form (data, target label) or (X, Y)

In [23]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()]
)

data_dir = "data/cloud_data" 
dataset = datasets.ImageFolder(root=data_dir, transform=transform) #Note. Add a transforms=transforms

batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
print(type(dataset))

<class 'torchvision.datasets.folder.ImageFolder'>


In [24]:
for data in dataloader:
    print(data)
    break

[tensor([[[[0.4667, 0.4627, 0.4627,  ..., 0.7020, 0.7020, 0.7059],
          [0.4745, 0.4706, 0.4667,  ..., 0.7098, 0.6980, 0.6980],
          [0.4745, 0.4745, 0.4667,  ..., 0.7216, 0.7098, 0.7020],
          ...,
          [0.0196, 0.0118, 0.0118,  ..., 0.0078, 0.0078, 0.0078],
          [0.0157, 0.0078, 0.0078,  ..., 0.0078, 0.0078, 0.0078],
          [0.0235, 0.0118, 0.0078,  ..., 0.0078, 0.0078, 0.0078]],

         [[0.4863, 0.4902, 0.4941,  ..., 0.7490, 0.7490, 0.7529],
          [0.4941, 0.4980, 0.5020,  ..., 0.7569, 0.7451, 0.7451],
          [0.4980, 0.5020, 0.5059,  ..., 0.7686, 0.7569, 0.7490],
          ...,
          [0.0314, 0.0196, 0.0196,  ..., 0.0000, 0.0000, 0.0000],
          [0.0275, 0.0157, 0.0157,  ..., 0.0000, 0.0000, 0.0000],
          [0.0353, 0.0235, 0.0157,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.6353, 0.6353, 0.6392,  ..., 0.7961, 0.7961, 0.8000],
          [0.6431, 0.6471, 0.6431,  ..., 0.8039, 0.7922, 0.7922],
          [0.6471, 0.6510, 0.6471,  ..., 

# Basic CNN

In [ ]:
class convNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) #Conv layer 1
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, )
